In [1]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import norm  # Import norm from scipy.stats
from datetime import datetime
import os  # Add this line to import the os module

# Load the CSV data into a dataframe (modify the filename/path accordingly)
filename = 'copy_spx_quotedata.csv'

# Reading the CSV file
df = pd.read_csv(filename, sep=",", header=None, skiprows=4)

# Assign the correct column names
df.columns = ['ExpirationDate','Calls','CallLastSale','CallNet','CallBid','CallAsk','CallVol',
              'CallIV','CallDelta','CallGamma','CallOpenInt','StrikePrice','Puts','PutLastSale',
              'PutNet','PutBid','PutAsk','PutVol','PutIV','PutDelta','PutGamma','PutOpenInt']

# Convert necessary columns to the correct data types
df['ExpirationDate'] = pd.to_datetime(df['ExpirationDate'], format='%a %b %d %Y')
df['StrikePrice'] = df['StrikePrice'].astype(float)
df['CallGamma'] = df['CallGamma'].astype(float)
df['PutGamma'] = df['PutGamma'].astype(float)
df['CallOpenInt'] = df['CallOpenInt'].astype(float)
df['PutOpenInt'] = df['PutOpenInt'].astype(float)

# Create a new column "Theo ES Price" using the new logic and round to the nearest 0.25
df['Theo ES Price'] = (df['StrikePrice'] *.0085) + df['StrikePrice'] ####You need to manually change the .0085 to get an accurate difference in ES and SPX
df['Theo ES Price'] = (df['Theo ES Price'] / 0.25).round() * 0.25

# Retrieve SPX spot price from yfinance
ticker = yf.Ticker("^GSPC")
spotPrice = ticker.history(period="1d")['Close'].iloc[-1]

# Define the ±350 range from the SPX Spot Price
minStrike = spotPrice - 350
maxStrike = spotPrice + 350

# Filter for options expiring today (0DTE)
todayDate = datetime.now()
df = df[df['ExpirationDate'].dt.date == todayDate.date()]

# ---=== Calculate TotalGamma ===---
df['CallGEX'] = df['CallGamma'] * df['CallOpenInt'] * 100 * spotPrice * spotPrice * 0.01
df['PutGEX'] = df['PutGamma'] * df['PutOpenInt'] * 100 * spotPrice * spotPrice * 0.01 * -1
df['TotalGamma'] = (df['CallGEX'] + df['PutGEX']) / 10**9

# ---=== Group data by StrikePrice and aggregate TotalGamma ===---
dfAgg = df.groupby(['StrikePrice']).agg({
    'TotalGamma': 'sum'
})

# Filter out rows with zero TotalGamma and within the 350-point range from SPX Spot Price
dfAgg_filtered = dfAgg[(dfAgg['TotalGamma'] != 0) & (dfAgg.index >= minStrike) & (dfAgg.index <= maxStrike)]
strikes_nonzero = dfAgg_filtered.index.values

# --- Derive Top 5 Highest and Lowest Strikes by Total Gamma ---
top_5 = dfAgg.nlargest(5, 'TotalGamma').reset_index()  # Get top 5 positive gamma (calls)
bottom_5 = dfAgg.nsmallest(5, 'TotalGamma').reset_index()  # Get bottom 5 negative gamma (puts)

# Assign "Call Gamma 1" to "Call Gamma 5" for the top 5 rows
top_5['GammaType'] = [f"Call Gamma {i+1}" for i in range(len(top_5))]

# Assign "Put Gamma 1" to "Put Gamma 5" for the bottom 5 rows
bottom_5['GammaType'] = [f"Put Gamma {i+1}" for i in range(len(bottom_5))]

# Combine top 5 and bottom 5 into a single DataFrame
top_bottom_5 = pd.concat([top_5, bottom_5])

# ---=== Calculate Gamma Flip Point (for fig2) ===---
fromStrike = 0.8 * spotPrice
toStrike = 1.2 * spotPrice
levels = np.linspace(fromStrike, toStrike, 60)

totalGamma = []

# Function to calculate gamma exposure for each level (used in fig2)
def calcGammaEx(S, K, vol, T, r, q, optType, OI):
    if vol == 0:
        return 0
    T = max(T, 1/365)  # Ensure T is not zero
    dp = (np.log(S/K) + (r - q + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    gamma = np.exp(-q*T) * norm.pdf(dp) / (S * vol * np.sqrt(T))
    return OI * 100 * S * S * 0.01 * gamma 

# Calculate Gamma Exposure for different index levels (used in fig2)
for level in levels:
    df['callGammaEx'] = df.apply(lambda row: calcGammaEx(level, row['StrikePrice'], row['CallIV'], 
                                                         0, 0, 0, "call", row['CallOpenInt']), axis=1)

    df['putGammaEx'] = df.apply(lambda row: calcGammaEx(level, row['StrikePrice'], row['PutIV'], 
                                                        0, 0, 0, "put", row['PutOpenInt']), axis=1)

    totalGamma.append(df['callGammaEx'].sum() - df['putGammaEx'].sum())

totalGamma = np.array(totalGamma) / 10**9

# Find Gamma Flip Point (zero gamma) for fig2
zeroCrossIdx = np.where(np.diff(np.sign(totalGamma)))[0]

if zeroCrossIdx.size > 0:
    negGamma = totalGamma[zeroCrossIdx]
    posGamma = totalGamma[zeroCrossIdx + 1]
    negStrike = levels[zeroCrossIdx]
    posStrike = levels[zeroCrossIdx + 1]
    zeroGamma = posStrike - ((posStrike - negStrike) * posGamma / (posGamma - negGamma))
    zeroGamma = zeroGamma[0]
else:
    zeroGamma = None
    print("Warning: No Gamma Flip Point found.")

# Select the relevant columns for the final CSV, including the new "Theo ES Price" column
final_df = top_bottom_5[['StrikePrice', 'TotalGamma', 'GammaType']]

# Now merge the 'Theo ES Price' column from the original DataFrame (df)
final_df = pd.merge(final_df, df[['StrikePrice', 'Theo ES Price']], on='StrikePrice', how='left')

# --- Add the "Gamma Flip" column ---
# Insert the Gamma Flip value in all rows
if zeroGamma is not None:
    final_df['Gamma Flip'] = zeroGamma
else:
    final_df['Gamma Flip'] = np.nan

# Define the output CSV filename with a full directory path
output_directory = r'C:\Users\Chris\OneDrive\Documents\NinjaTrader 8\bin\Custom'  # Modify this path to your desired directory
if not os.path.exists(output_directory):
    os.makedirs(output_directory)  # Create the directory if it doesn't exist

# Combine the directory and the filename
csv_filename = os.path.join(output_directory, 'top_bottom_gamma_exposure.csv')

# Save the final DataFrame to a CSV file
final_df.to_csv(csv_filename, index=False)
print(f"Top 5 and Bottom 5 Gamma Exposure data with Theo ES Price and Gamma Flip saved to {csv_filename}")


# ---=== First Figure: Absolute Gamma Exposure (Filtered and Range-Restricted) ===---
fig1 = go.Figure()

# Add bar trace for Gamma Exposure (filtered and restricted to the range)
fig1.add_trace(go.Bar(
    x=strikes_nonzero,
    y=dfAgg_filtered['TotalGamma'].to_numpy(),
    marker=dict(color='blue', line=dict(color='black', width=1)),
    name='Gamma Exposure',
))

# Add a vertical line for the SPX spot price
fig1.add_trace(go.Scatter(
    x=[spotPrice, spotPrice],
    y=[min(dfAgg_filtered['TotalGamma'].to_numpy()), max(dfAgg_filtered['TotalGamma'].to_numpy())],
    mode="lines",
    line=dict(color="red", width=2),
    name=f"SPX Spot: {spotPrice:.0f}"
))

# Update layout for the first plot (set x-axis range between minStrike and maxStrike)
fig1.update_layout(
    title=f"Total Gamma: ${df['TotalGamma'].sum():.2f} Bn per 1% SPX Move (0DTE)",
    xaxis_title='Strike',
    yaxis_title='Spot Gamma Exposure ($ billions/1% move)',
    xaxis=dict(range=[minStrike, maxStrike]),  # Limit x-axis range
    showlegend=True
)

# ---=== Second Figure: Gamma Exposure Profile (0DTE with Full Range) ===---

# Define levels (price range for the x-axis)
fromStrike = 0.8 * spotPrice
toStrike = 1.2 * spotPrice
levels = np.linspace(fromStrike, toStrike, 60)

totalGamma = []

# Function to calculate gamma exposure for each level
def calcGammaEx(S, K, vol, T, r, q, optType, OI):
    if vol == 0:
        return 0
    T = max(T, 1/365)  # Ensure T is not zero
    dp = (np.log(S/K) + (r - q + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    gamma = np.exp(-q*T) * norm.pdf(dp) / (S * vol * np.sqrt(T))
    return OI * 100 * S * S * 0.01 * gamma 

# Calculate Gamma Exposure for different index levels
for level in levels:
    df['callGammaEx'] = df.apply(lambda row: calcGammaEx(level, row['StrikePrice'], row['CallIV'], 
                                                         0, 0, 0, "call", row['CallOpenInt']), axis=1)

    df['putGammaEx'] = df.apply(lambda row: calcGammaEx(level, row['StrikePrice'], row['PutIV'], 
                                                        0, 0, 0, "put", row['PutOpenInt']), axis=1)

    totalGamma.append(df['callGammaEx'].sum() - df['putGammaEx'].sum())

totalGamma = np.array(totalGamma) / 10**9

# Find Gamma Flip Point (the point where totalGamma crosses zero)
zeroCrossIdx = np.where(np.diff(np.sign(totalGamma)))[0]

# Check if there's any zero-crossing index
if zeroCrossIdx.size > 0:
    negGamma = totalGamma[zeroCrossIdx]
    posGamma = totalGamma[zeroCrossIdx + 1]
    negStrike = levels[zeroCrossIdx]
    posStrike = levels[zeroCrossIdx + 1]

    zeroGamma = posStrike - ((posStrike - negStrike) * posGamma / (posGamma - negGamma))
    zeroGamma = zeroGamma[0]
else:
    # Handle the case where no zero-crossing is found
    zeroGamma = None
    print("Warning: No Gamma Flip Point found. Skipping the Gamma Flip line in the chart.")

# --- Create the Gamma Exposure Profile Plot (Full Range) ---
fig2 = go.Figure()

# Add line trace for Gamma Exposure Profile
fig2.add_trace(go.Scatter(
    x=levels,
    y=totalGamma,
    mode='lines',
    line=dict(color='blue'),
    name='0DTE Gamma Exposure'
))

# Add vertical line for the SPX spot price
fig2.add_trace(go.Scatter(
    x=[spotPrice, spotPrice],
    y=[min(totalGamma), max(totalGamma)],
    mode="lines",
    line=dict(color="red", width=2),
    name=f"SPX Spot: {spotPrice:.0f}"
))

# Add vertical line for Gamma Flip point if found
if zeroGamma is not None:
    fig2.add_trace(go.Scatter(
        x=[zeroGamma, zeroGamma],
        y=[min(totalGamma), max(totalGamma)],
        mode="lines",
        line=dict(color="black", width=2),
        name=f"Gamma Flip: {zeroGamma:.0f}"
    ))

# Fill areas with positive and negative gamma exposure
fig2.add_shape(
    type="rect",
    x0=fromStrike, y0=min(totalGamma),
    x1=zeroGamma if zeroGamma is not None else spotPrice,
    y1=max(totalGamma),
    fillcolor="red",
    opacity=0.1,
    line_width=0,
)

fig2.add_shape(
    type="rect",
    x0=zeroGamma if zeroGamma is not None else spotPrice,
    y0=min(totalGamma),
    x1=toStrike, y1=max(totalGamma),
    fillcolor="green",
    opacity=0.1,
    line_width=0,
)

# Update layout for the second plot (with full range)
fig2.update_layout(
    title=f"Gamma Exposure Profile, SPX (0DTE), {todayDate.strftime('%d %b %Y')}",
    xaxis_title='Index Price',
    yaxis_title='Gamma Exposure ($ billions/1% move)',
    xaxis=dict(range=[fromStrike, toStrike]),  # Full range (no restriction to ±350)
    showlegend=True
)

# ---=== Third Figure: Top 10 Highest and Lowest Gamma Exposure Levels (0DTE) ===---

# Sort and select top 10 highest and lowest gamma exposures along with strike prices
top_10 = dfAgg.nlargest(10, 'TotalGamma').reset_index()  # Reset index to keep the 'StrikePrice' column
bottom_10 = dfAgg.nsmallest(10, 'TotalGamma').reset_index()  # Reset index to keep the 'StrikePrice' column

# Combine top 10 and bottom 10 into a single DataFrame
combined_gamma = pd.concat([top_10, bottom_10])

# Filter the combined data to only include strikes within ±350 of SPX Spot Price
combined_gamma_filtered = combined_gamma[(combined_gamma['StrikePrice'] >= minStrike) & (combined_gamma['StrikePrice'] <= maxStrike)]

# --- Save Top 10 and Lowest 10 to a CSV File ---
csv_filename = 'top_bottom_gamma_exposure.csv'
combined_gamma_filtered.to_csv(csv_filename, index=False)
print(f"Top 10 and Lowest 10 Strike levels saved to {csv_filename}")

# --- Create the Top 10 Gamma Exposure Levels Plot ---
fig3 = go.Figure()

# Add bar trace for all strikes within the range
fig3.add_trace(go.Bar(
    x=dfAgg_filtered.index,
    y=dfAgg_filtered['TotalGamma'],
    marker=dict(color='blue', line=dict(color='black', width=1)),
    name='Gamma Exposure',
))

# Highlight top 10 highest gamma exposures within the range
for i, row in top_10.iterrows():
    if minStrike <= row['StrikePrice'] <= maxStrike:
        fig3.add_trace(go.Scatter(
            x=[row['StrikePrice']], y=[row['TotalGamma']],
            mode='markers+text',
            marker=dict(color='green', size=10),
            name=f"Top Strike: {row['StrikePrice']:.0f}"
        ))

# Highlight top 10 lowest gamma exposures within the range
for i, row in bottom_10.iterrows():
    if minStrike <= row['StrikePrice'] <= maxStrike:
        fig3.add_trace(go.Scatter(
            x=[row['StrikePrice']], y=[row['TotalGamma']],
            mode='markers+text',
            marker=dict(color='blue', size=10),
            name=f"Low Strike: {row['StrikePrice']:.0f}"
        ))

# Add a vertical line for the SPX spot price
fig3.add_trace(go.Scatter(
    x=[spotPrice, spotPrice],
    y=[min(dfAgg_filtered['TotalGamma']), max(dfAgg_filtered['TotalGamma'])],
    mode="lines",
    line=dict(color="red", width=2),
    name=f"SPX Spot: {spotPrice:.0f}"
))

# Update layout for the third plot (set x-axis range between minStrike and maxStrike)
fig3.update_layout(
    title=f"Top 10 Highest and Lowest Gamma Exposure Levels (0DTE)",
    xaxis_title='Strike',
    yaxis_title='Spot Gamma Exposure ($ billions/1% move)',
    xaxis=dict(range=[minStrike, maxStrike]),
    showlegend=True
)

# Show all three figures
fig1.show()
fig2.show()
fig3.show()

Top 5 and Bottom 5 Gamma Exposure data with Theo ES Price and Gamma Flip saved to C:\Users\Chris\OneDrive\Documents\NinjaTrader 8\bin\Custom\top_bottom_gamma_exposure.csv
Top 10 and Lowest 10 Strike levels saved to top_bottom_gamma_exposure.csv
